# BERT senetence similarty analyser 
by using BERT senetence similarty analyser we can begin to get a better understanding of if a question is close enoough to a target intent to use the corasponding answer.


## Step 1
lets start by reading in the current data we have set inside a target json file

In [1]:
import json
  
# Opening JSON file
f = open('intents.json',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
for i in data['intents']:
    print(i)
  
# Closing file
f.close()

{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], 'responses': ['Hello!'], 'context_set': ''}
{'tag': 'goodbye', 'patterns': ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], 'responses': ['Goodbye!'], 'context_set': ''}
{'tag': 'what', 'patterns': ['what are you', 'who are you', 'what do you do', 'what is your purpose', 'what is this'], 'responses': ['I am a USCIS chat bot for handeling questions and customer service'], 'context_set': ''}
{'tag': 'USCIS', 'patterns': ['what is USCIS', 'USCIS dose what?', 'whats USCIS do?'], 'responses': ["U.S. Citizenship and Immigration Services is an agency of the United States Department of Homeland Security that administers the country's naturalization and immigration system"], 'context_set': ''}
{'tag': 'DACA', 'patterns': ['What is purpose of DACA?', 'What is deferred action?', 'What is DACA?'], 'responses': ['Under existing regulations, an individual whose case has 

## Step 2
Now that we have our data lets orginize it so that we have all our intent questions and answers alined 

In [2]:
questions =[]
answers = []

for i in data['intents']:
    questions.append(i['patterns'])
    answers.append(i['responses'][0])
    
print(questions)
print('')
print('')
print(answers)

[['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Whats up'], ['cya', 'See you later', 'Goodbye', 'I am Leaving', 'Have a Good day'], ['what are you', 'who are you', 'what do you do', 'what is your purpose', 'what is this'], ['what is USCIS', 'USCIS dose what?', 'whats USCIS do?'], ['What is purpose of DACA?', 'What is deferred action?', 'What is DACA?'], ['Why is DACA ending?', 'Why is DACA phasing out', 'How can I get more DACA information?', 'Frequently asked DACA questions'], ['when are you open', 'what are your hours', 'hours of operation', 'what is your availability']]


['Hello!', 'Goodbye!', 'I am a USCIS chat bot for handeling questions and customer service', "U.S. Citizenship and Immigration Services is an agency of the United States Department of Homeland Security that administers the country's naturalization and immigration system", 'Under existing regulations, an individual whose case has been deferred is eligible to receive employment authorization for the 

## Step 3
Now that we have our data set in two simple lists we can run comparisons on a question type versus intents and return the answer that makes the most sence given BERTS understanding of our target question.

First we will set up the BERT text similarity tracker - 

In [3]:
#use this command to install sentence transformers if you need them 
#!pip install sentence-transformers

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('bert-base-nli-mean-tokens')


## Step 4
Now that we have that set up build a method that can take in the data and map comparisons 

In [4]:
def get_similaritys(text, comparison_list, index):
    
    sentences = [text] + comparison_list    
    sentence_embeddings = model.encode(sentences)
    sentence_embeddings.shape
    
    #Let's calculate cosine similarity for sentence 0:
    result=cosine_similarity([sentence_embeddings[0]],sentence_embeddings[1:])
    
    Max = max(result[0])
    
    if(Max > 0.9):
        return ['Bingo',index]  
    return [Max, index]
        

In [5]:
# test the method 
targets =['this is a good dog.', 'the cat is free.', 'who could have predicted this?','time is older than us']
text='i think dogs are cool'

result = get_similaritys(text, targets, 0)

print(result)

[0.89871323, 0]


In [6]:
questions1 = [['this is a good dog.', 'the dog is great'],['this cat is free','a free cat walks free'],['who predicted this?'],['time is old','time is older than us']]
answers1 =['indeed a good dog','free cats for all','not me thats for sure ', 'yes and we all should feel time is the master']


## Step 5
Now that we have a comparison method we can build a central model input method that will respond to a question with a matching answer or if none seem likely from the question a simple sorry please repeat that I do not understand.

In [7]:
def qa_model_input(text, questions1, answers1):
    index =-1
    targets =[]
    for x in questions1:
        index +=1
        result = get_similaritys(text, x, 0)
        if(result[0]=='Bingo'):
            return answers1[index]
        else:
            targets.append(result)
    max_value = max(targets) 
    if(max_value[0] < 0.85):
        return("sorry please repeat that I do not understand.")
    max_index = targets.index(max_value)
    return answers1[index] 

In [8]:
# Test run
run = True
while run:
    val = input("Hey Im a chat bot what can I help you with today : ")
    if(val =='/exit' or val == 'no'):
        print('good bye lets talk again soon!')
        break
    
    result = qa_model_input(val, questions1, answers1)
    print('Question : \n')
    print(val)
    print('Answer : \n')
    print(result)
    print('\n')
    print('Great question care to ask another ?')
    print('\n')
    

Hey Im a chat bot what can I help you with today : /exit
good bye lets talk again soon!


In [9]:
## max wait time is 4 to 6 seconds currently
run = True
while run:
    val = input("Hello Im the Maxmimus help desk chat bot what can I help you with today : ")
    if(val =='/exit' or val == 'no'):
        print('good bye lets talk again soon!')
        break
    
    result = qa_model_input(val, questions, answers)
    print('Question : \n')
    print(val)
    print('Answer : \n')
    print(result)
    print('\n')
    print('Great question care to ask another ?')
    print('\n')

Hello Im the Maxmimus help desk chat bot what can I help you with today : DACA?
Question : 

DACA?
Answer : 

Under existing regulations, an individual whose case has been deferred is eligible to receive employment authorization for the period of deferred action, provided he or she can demonstrate an economic necessity for employment. DHS


Great question care to ask another ?


Hello Im the Maxmimus help desk chat bot what can I help you with today : /exit
good bye lets talk again soon!


In [10]:
# result = qa_model_input(val, questions, answers)

# set up parrot to do parra phrases 
# Import libraries
from parrot import Parrot
import torch
import warnings
warnings.filterwarnings("ignore")
print('parrot loaded ready to work')


parrot loaded ready to work


In [11]:
##Init models (make sure you init ONLY once if you integrate this to your code)
parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

In [12]:
#build compresed method 
def get_para(phrase, num, length):
    texts = []
    para_phrases = parrot.augment(input_phrase=phrase, use_gpu=False, diversity_ranker='levenshtein', do_diverse=False,
                                max_return_phrases=num, max_length=length, adequacy_threshold=0.9, fluency_threshold=0.9)
    for para_phrase in para_phrases:
        text = para_phrase[0]
        texts.append(text)
    
    return texts
        

text = 'Whats the most delicious papayas?'
texts = get_para(text, 10, len(text)+5)
print(texts)

['tell me the best papaya of all time?', 'tell me the best papaya?', 'list some of the best papayas?', 'list some of the best tasty papayas?', 'which are the best papayas?', 'tell me the most addictive papaya?', 'list some of the most delicious papayas?']


In [13]:
# now run all intents and make 0 succsess 1 fail
results = []

index =0
for x in questions:
    answer = answers[index]
    for i in x:
        texts = get_para(i, 10, len(text)+5)
        texts.append(i)
        print(texts)
        for text in texts:
            res = qa_model_input(text, questions, answers)
            #print('AI :', res)
            #print('answer :', answer)
            if(answer == res):
                results.append(0)
            else:
                results.append(1)
    index +=1
    print(results)

print('')
print('these are the final results')
print(results)

['Hi', 'Hi']
["how's your day?", 'how are you doing?', 'how are you?', 'How are you']
['is anyone around?', 'who is there?', 'can you name anyone else?', 'is anyone out there?', 'is anyone in there?', 'is there anyone there?', 'Is anyone there?']
['Hello', 'Hello']
['good day', 'Good day']
['what are your thoughts on it?', 'what do you want to know?', 'tell me the problem?', "what's up i guess?", "what's going on?", "what's up?", 'whats up?', 'what was up?', 'Whats up']
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0]
['cya', 'cya']
['see you later', "we'll see you later", "let's see you later", 'See you later']
['goodbye', 'soooooooo goodbye', "let's say goodbye", 'Goodbye']
["i'll leave", "i'm leaving", 'I am Leaving']
['happy day', 'have a good day', 'Have a Good day']
[0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
['what is your identity?', 'what do you have?', 'what are yo

## Save the data to CSV

now we take the results and save them to a csv format 


In [14]:
import csv

header = ['index', 'pred', 'expected']
data = []

index = 0
for x in results:
    item = [index,x,0]
    data.append(item)
    index +=1
    
    

with open('csv data/BERT_data.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

print('done all data writen to csv')

done all data writen to csv
